## 1. Data Description
<ul>
<li>there are 809 training examples</li>
<li>each of them is a dataframe with time, mass , and intensity values</li>
<li>dataframe lenghts are variable</li>
</ul>


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# for processing numbers and files csv
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, log_loss

# model related
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#for gpu

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10163601398610753540
xla_global_id: -1
]


False

In [ ]:
# for working in kaggle use this paths
kag_path_in = "/kaggle/input/mars-spec-train/"
kag_path_out = "/kaggle/working/"

kag_path_in = "/content/drive/MyDrive/" # google drive
# add custom paths here (uncomment)
# kag_path_in = ""
kag_path_out = ""

In [ ]:
# metadata = pd.read_csv('/kaggle/input/mars-spec-train/metadata.csv').fillna(0)
# train_labels = pd.read_csv('/kaggle/input/mars-spec-train/train_labels.csv')
metadata = pd.read_csv(kag_path_in+'metadata.csv').fillna(0)
train_labels = pd.read_csv(kag_path_in+'train_labels.csv')

## 2. Preprocessing and Feature extraction
<ul>
<li>mass values are grouped into values seperated by 0.50 </li>
<li>time values are grouped into values seperated by 0.02 </li>
<li>Time * Mass matrix (2700,1400) with Intensity values </li>
</ul>

In [ ]:
# constants
num_mass_values = 1400
num_time_values = 2700
matrix_shape = (num_time_values, num_mass_values)

In [ ]:

# Function to create the Time X Mass matrix for a single data sample
def create_matrix(id):

    sample_id, split, derivate, path, _ = metadata.iloc[id]
    data = pd.read_csv(kag_path_in+path)

    label = None
    if (split == 'train'):
        label = train_labels.iloc[id].values[1:]

    # discretization of time and mass
    data.time = (data.time*50).round()
    data.mass = (data.mass*2).round()
    data.time = data.time.astype(int)
    data.mass = data.mass.astype(int)

    # scaling intensity
    data.intensity = MinMaxScaler().fit_transform(data.intensity.values.reshape(-1, 1))

    # filling matrix
    matrix = np.zeros(matrix_shape)
    matrix[data['time'],data['mass']] = data['intensity']

    return matrix , label


## 3. Data Generation
<ul>
<li>Storing data matrix is costly</li>
<li>data is generated in batches</li>
</ul>


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
    def __len__(self):
        return len(self.data) // self.batch_size

    def __getitem__(self, idx):
        low = idx * self.batch_size
        high = min(low + self.batch_size, len(self.data))
        batch_x = []
        batch_y = []

        for i, data_sample in enumerate(self.data[low:high]):
            processed_sample, processed_label = self.preprocess_data(data_sample)
            batch_x.append(processed_sample)
            batch_y.append(processed_label)


        while len(batch_x) < self.batch_size:
            random_no = np.random.randint(0,len(self.data))
            processed_sample, processed_label = self.preprocess_data(data_sample)
            batch_x.append(processed_sample)
            batch_y.append(processed_label)

        return np.array(batch_x), np.array(batch_y)

    def preprocess_data(self, data_sample):
        matrix, label = create_matrix(data_sample)
        if type(label) == np.ndarray:
            label = np.array(label).astype(int)
        return matrix , label


## 4. Model
<ul>
<li>convolutional layers </li>
<li>pooling layers (max pooling) </li>
<li>fully connected layers</li>
<li>sigmoid layer</li>
<li>output positive label probability</li>
</ul>

In [ ]:
label_names = [col for col in train_labels.columns[1:]]

In [ ]:
# contants
batch_size = 16
num_epochs = 75
model = None
history = None

In [ ]:
training_data = range(809)
train_data, val_data = train_test_split(training_data, test_size=0.2, random_state=42)
train_data_generator = DataGenerator(training_data, batch_size)
val_data_generator = DataGenerator(val_data, batch_size)
print(len(train_data), len(val_data))

647 162


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (120,1), padding= 'same',strides = (100,1),activation='relu', input_shape=(num_time_values, num_mass_values, 1)))
model.add(MaxPooling2D(pool_size=(9, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(9, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=opt, metrics=['AUC']) # from logits = true

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 27, 1400, 32)      3872      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 700, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 67200)             0         
                                                                 
 dense (Dense)               (None, 128)               8601728   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                        

In [ ]:
es = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = True  )
history = model.fit(train_data_generator, epochs=num_epochs, validation_data=val_data_generator, callbacks=[es],verbose = 1)

Epoch 1/75
50/50 [==============================] - 818s 16s/step - loss: 0.3037 - auc: 0.7781 - val_loss: 0.1948 - val_auc: 0.9138
Epoch 2/75
50/50 [==============================] - 604s 12s/step - loss: 0.2052 - auc: 0.9062 - val_loss: 0.1740 - val_auc: 0.9505
Epoch 3/75
50/50 [==============================] - 556s 11s/step - loss: 0.1774 - auc: 0.9341 - val_loss: 0.1378 - val_auc: 0.9656
Epoch 4/75
48/50 [===========================>..] - ETA: 18s - loss: 0.1496 - auc: 0.9565

In [ ]:
model.save("cnn_model_test.keras")

## 5. Performance
<ul>
<li>train and validation losses</li>
</ul>

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.legend()
plt.show()



In [ ]:
y_pred = model.predict(DataGenerator(range(809),1), verbose = 0)

### Classification Reports

In [ ]:
for i in range(9):
    y_true_column = train_labels.iloc[:, i+1].values
    y_pred_column = y_pred[:, i].round()

    report = classification_report(y_true_column, y_pred_column)
    print(f"{label_names[i]}")
    print(report)
    print(f"roc_auc_score : {roc_auc_score(y_true_column, y_pred[:, i])}")
    print(f"log loss      : {log_loss(y_true_column, y_pred[:, i])}")

    print("-------------------------------------------------------------")
